In [204]:
import pandas as pd
import numpy as np
pd.options.display.max_rows = 999
pd.set_option('display.max_columns', 500)

In [205]:
a_rsv = pd.read_csv('data/air_reserve.csv')
a_info = pd.read_csv('data/air_store_info.csv')
a_visit = pd.read_csv('data/air_visit_data.csv')
date_info = pd.read_csv('data/date_info.csv')
h_rsv = pd.read_csv('data/hpg_reserve.csv')
h_info = pd.read_csv('data/hpg_store_info.csv')
store_id = pd.read_csv('data/store_id_relation.csv')

# Data Cleaning
a_rsv : air store reservation (314 stores)
a_info : air store info(store name, genre, geo)
a_visit : sotre visit per day (only 829 store...)
date_info : holiday information, holiday flag and day of week
h_rsv : hpg store reservation  (13325 stores)
h_info : hpg store info (store name, genre, geo)
sotre_id : matching air store with hpg store...(150 has both air and hpg)

Steps

1.make the base `full_temp` table containing each calendar date for each restaurant.

2.get `full` table containing total reservation（hpg and air）, store info, calender info, and target value - store visits

Note that this table is based on the whole period(training + testing), from 2016 Jan 1 to 2017 May 31
The visitor info ends in 2017-4-22, However, the reservation info has all the way to 2017-5-31

3.add `dayofweek`(`day_of_week` is string), `day`, `month`, `year` and `dayofyear` columns  on `full` table

4.add `closed_flag`, indicating if the store closed, `{0:open, 1:closed, 2:not_listed(some of the store listed on AIR system recently)}`, on `full` table

Note that 2016-01-01 to 2016-01-03 are national holiday break, a lot of restaurants closed at that time, therefore many restaurants has `earlist_open_date` between 2016-01-01 to 2016-01-06. We later make some changes to these restaurant `closed_flag` column, view those days as closed instead of not_listed

### step 1

In [206]:
# get a list of all stores
store_id_list = a_visit.air_store_id.unique()
a_visit.air_store_id.nunique()

829

In [207]:
# get a list of all dates
date_list = date_info.calendar_date.unique()
date_info['calendar_date'].nunique()

517

In [208]:
# get cartesian product
index = pd.MultiIndex.from_product([store_id_list, date_list], names = ["air_store_id", "calendar_date"])
full_temp = pd.DataFrame(index = index).reset_index()

We have 829 stores and 517 dates in total, we should have 829 * 517 = 428,593 rows.

In [209]:
full_temp.shape

(428593, 2)

### step2

In [215]:
full_temp.head()

,air_store_id,calendar_date,day_of_week,holiday_flg,air_genre_name,air_area_name,latitude,longitude,merge_on,visit_date,reserve_visitors
0,air_ba937bf13d40fb24,2016-01-01,Friday,1,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,2016-01-01,NaN,NaN
1,air_ba937bf13d40fb24,2016-01-02,Saturday,1,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,2016-01-02,NaN,NaN
2,air_ba937bf13d40fb24,2016-01-03,Sunday,1,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,2016-01-03,NaN,NaN
3,air_ba937bf13d40fb24,2016-01-04,Monday,0,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,2016-01-04,NaN,NaN
4,air_ba937bf13d40fb24,2016-01-05,Tuesday,0,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,2016-01-05,NaN,NaN


In [211]:
# add date and holiday info
full_temp = full_temp.merge(date_info, on = 'calendar_date')

In [212]:
# add AIR store info 
full_temp = full_temp.merge(a_info, on = 'air_store_id')

In [213]:
# add AIR reservation info
a_rsv['visit_datetime']= pd.to_datetime(a_rsv['visit_datetime']) 
a_rsv['visit_date'] = a_rsv['visit_datetime'].dt.date
air_temp = a_rsv.groupby(['air_store_id', 'visit_date']).agg({'reserve_visitors':'sum'}).reset_index()

In [214]:
air_temp['merge_on']= pd.to_datetime(air_temp['visit_date']) 
full_temp['merge_on']= pd.to_datetime(full_temp['calendar_date'])
full_temp = full_temp.merge(air_temp, on = ['air_store_id', 'merge_on'], how = 'left')

In [218]:
# add in air_store and hpg_store realtionship column
full_temp = full_temp.merge(store_id, on = 'air_store_id', how = 'left')

In [219]:
# add in HPG reservation info 
h_rsv['visit_datetime']= pd.to_datetime(h_rsv['visit_datetime']) 
h_rsv['visit_date'] = h_rsv['visit_datetime'].dt.date
hpg_temp = h_rsv.groupby(['hpg_store_id', 'visit_date']).agg({'reserve_visitors':'sum'}).reset_index()
# full_temp.merge(h_rsv[['hpg_store_id', 'reserve_visitors']], on = )

In [220]:
hpg_temp['merge_on']= pd.to_datetime(hpg_temp['visit_date']) 
full_temp['merge_on']= pd.to_datetime(full_temp['calendar_date'])
full_temp = full_temp.merge(hpg_temp, on = ['hpg_store_id', 'merge_on'], how = 'left', suffixes = ('_air','_hpg'))

In [221]:
full_temp.hpg_store_id.value_counts() # sanity check

hpg_08def9f764fcca4c    517
hpg_47c5b2b16a840e89    517
hpg_874415e6e7ccfe13    517
hpg_7664eec3cd802c91    517
hpg_381bc6f038ec3ea8    517
hpg_8072ff2fb418e273    517
hpg_6507818140022452    517
hpg_43a5666d1b6fa70b    517
hpg_4dabe28f13a5b993    517
hpg_0f78ef8d1e4b23d8    517
hpg_cd98b0cad88f7123    517
hpg_653238a84804d8e7    517
hpg_053676b196ae1e1e    517
hpg_7459535472b27200    517
hpg_583dddac1a035048    517
hpg_e29b78a4955b72f5    517
hpg_40aff6385800ebb1    517
hpg_4ca41b70c41bc844    517
hpg_ee424bb170e133f1    517
hpg_7dae95f9d3db1734    517
hpg_dad8da3ff443ba1a    517
hpg_878cc70b1abc76f7    517
hpg_03795a62e1c5f1c3    517
hpg_623477d346cf1822    517
hpg_1f9643b5673938d9    517
hpg_945c2807cef88022    517
hpg_a85b37424291216b    517
hpg_33df7b58b0f9d93f    517
hpg_0a587f3924c8093c    517
hpg_3347c0399df81142    517
hpg_b0cbb707b862dde7    517
hpg_2d4200d0ebdb4946    517
hpg_3ebbe4996edd55d5    517
hpg_fa4eeada9ad7b59f    517
hpg_f83413cde79ac5e7    517
hpg_71765891f7a4ac62

In [222]:
# finally, bring in the most important column - visitors
a_visit['merge_on']= pd.to_datetime(a_visit['visit_date']) 
full = full_temp.merge(a_visit[['air_store_id', 'merge_on', 'visitors']], on = ['air_store_id', 'merge_on'], how = 'left')

In [223]:
full_temp.head(2)

,air_store_id,calendar_date,day_of_week,holiday_flg,air_genre_name,air_area_name,latitude,longitude,merge_on,visit_date_air,reserve_visitors_air,hpg_store_id,visit_date_hpg,reserve_visitors_hpg
0,air_ba937bf13d40fb24,2016-01-01,Friday,1,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,2016-01-01,NaN,NaN,NaN,NaN,NaN
1,air_ba937bf13d40fb24,2016-01-02,Saturday,1,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,2016-01-02,NaN,NaN,NaN,NaN,NaN


### step3
add `dayofweek`(`day_of_week` is string), `day`, `month`, `year` and `dayofyear` columns  on `full` table

In [224]:
full['day'] = full['merge_on'].dt.day
full['month'] = full['merge_on'].dt.month
full['year'] = full['merge_on'].dt.year
full['dayofweek'] = full['merge_on'].dt.weekday # Monday is 0 and Sunday is 6.
full['dayofyear'] = full['merge_on'].dt.dayofyear

### step4
add `closed_flag`, indicating if the store closed, `{0:open, 1:closed, 2:not_listed(some of the store opens recently, don't have full 2-year data)}`, on `full` table

In [225]:
a_visit.head()

,air_store_id,visit_date,visitors,merge_on
0,air_ba937bf13d40fb24,2016-01-13,25,2016-01-13
1,air_ba937bf13d40fb24,2016-01-14,32,2016-01-14
2,air_ba937bf13d40fb24,2016-01-15,29,2016-01-15
3,air_ba937bf13d40fb24,2016-01-16,22,2016-01-16
4,air_ba937bf13d40fb24,2016-01-18,6,2016-01-18


In [226]:
# get the earlist open day for each store.
earlist_open_series = a_visit.groupby('air_store_id')['visit_date'].min()

In [227]:
full = full.merge(earlist_open_series.rename('earlist_open_date'), left_on = 'air_store_id', right_index=True)

In [228]:
# write a function to decide if the store is closed/open/not_listed 
def cal_closed_flag(row):
    #result = 0
    if row['earlist_open_date'] > row['calendar_date']:
        result = 2
    else: # now the store opens/listed in AIR system, we can look at which the store close
        if row.visitors >= 0:
            result = 0
        else:
            result = 1
    return result

In [229]:
full['closed_flag'] = full.apply(cal_closed_flag, axis=1)

In [230]:
full.earlist_open_date.value_counts().head(15)

2016-07-01    234718
2016-01-04     41877
2016-01-05     39292
2016-01-01     24816
2016-01-02     16027
2016-01-03     10340
2016-01-06     10340
2016-07-02      7238
2016-01-08      4136
2016-01-07      3102
2016-07-03      2068
2016-07-05      1551
2016-01-09      1551
2016-03-25      1034
2016-07-04      1034
Name: earlist_open_date, dtype: int64

We can see that the top common earlist open day is from 2016-01-01 to 2016-01-06. However, this is not because that the restaurants start listing on the AIR system from those days only. Because 2016-01-01 to 2016-01-03 are the official holiday break, pretty sure a lot of restaurants would close at those days naturely. 

Therefore, we need to make some changes to the `closed_flag` for these restaurants.

In [231]:
# make some changes.
def cal_closed_flag_2(row):
    if row['earlist_open_date'] \
       in ['2016-01-01', '2016-01-02', '2016-01-03', '2016-01-04', '2016-01-05', '2016-01-06'] and \
       row['calendar_date'] < row['earlist_open_date'] :
        result = 1 # these are listed restaurants but not open on nathional holiday. 
    else:
        if row['earlist_open_date'] > row['calendar_date']:
            result = 2
        else: # now the store opens/listed in AIR system, we can look at when the store close
            if row.visitors >= 0:
                result = 0
            else:
                result = 1
    return result

In [232]:
full['closed_flag'] = full.apply(cal_closed_flag_2, axis=1)

In [233]:
# sanity check
full[(full.air_store_id == 'air_683371d9baabf410')].head(10)

,air_store_id,calendar_date,day_of_week,holiday_flg,air_genre_name,air_area_name,latitude,longitude,merge_on,visit_date_air,reserve_visitors_air,hpg_store_id,visit_date_hpg,reserve_visitors_hpg,visitors,day,month,year,dayofweek,dayofyear,earlist_open_date,closed_flag
79618,air_683371d9baabf410,2016-01-01,Friday,1,Japanese food,Hyōgo-ken Himeji-shi Yasuda,34.815149,134.685353,2016-01-01,NaN,NaN,hpg_7dae95f9d3db1734,NaN,NaN,NaN,1,1,2016,4,1,2016-01-06,1
79619,air_683371d9baabf410,2016-01-02,Saturday,1,Japanese food,Hyōgo-ken Himeji-shi Yasuda,34.815149,134.685353,2016-01-02,NaN,NaN,hpg_7dae95f9d3db1734,NaN,NaN,NaN,2,1,2016,5,2,2016-01-06,1
79620,air_683371d9baabf410,2016-01-03,Sunday,1,Japanese food,Hyōgo-ken Himeji-shi Yasuda,34.815149,134.685353,2016-01-03,NaN,NaN,hpg_7dae95f9d3db1734,NaN,NaN,NaN,3,1,2016,6,3,2016-01-06,1
79621,air_683371d9baabf410,2016-01-04,Monday,0,Japanese food,Hyōgo-ken Himeji-shi Yasuda,34.815149,134.685353,2016-01-04,NaN,NaN,hpg_7dae95f9d3db1734,NaN,NaN,NaN,4,1,2016,0,4,2016-01-06,1
79622,air_683371d9baabf410,2016-01-05,Tuesday,0,Japanese food,Hyōgo-ken Himeji-shi Yasuda,34.815149,134.685353,2016-01-05,NaN,NaN,hpg_7dae95f9d3db1734,NaN,NaN,NaN,5,1,2016,1,5,2016-01-06,1
79623,air_683371d9baabf410,2016-01-06,Wednesday,0,Japanese food,Hyōgo-ken Himeji-shi Yasuda,34.815149,134.685353,2016-01-06,NaN,NaN,hpg_7dae95f9d3db1734,NaN,NaN,8.0,6,1,2016,2,6,2016-01-06,0
79624,air_683371d9baabf410,2016-01-07,Thursday,0,Japanese food,Hyōgo-ken Himeji-shi Yasuda,34.815149,134.685353,2016-01-07,NaN,NaN,hpg_7dae95f9d3db1734,NaN,NaN,21.0,7,1,2016,3,7,2016-01-06,0
79625,air_683371d9baabf410,2016-01-08,Friday,0,Japanese food,Hyōgo-ken Himeji-shi Yasuda,34.815149,134.685353,2016-01-08,NaN,NaN,hpg_7dae95f9d3db1734,NaN,NaN,65.0,8,1,2016,4,8,2016-01-06,0
79626,air_683371d9baabf410,2016-01-09,Saturday,0,Japanese food,Hyōgo-ken Himeji-shi Yasuda,34.815149,134.685353,2016-01-09,NaN,NaN,hpg_7dae95f9d3db1734,NaN,NaN,28.0,9,1,2016,5,9,2016-01-06,0
79627,air_683371d9baabf410,2016-01-10,Sunday,0,Japanese food,Hyōgo-ken Himeji-shi Yasuda,34.815149,134.685353,2016-01-10,NaN,NaN,hpg_7dae95f9d3db1734,2016-01-10,20.0,42.0,10,1,2016,6,10,2016-01-06,0


In [234]:
full.dtypes

air_store_id                    object
calendar_date                   object
day_of_week                     object
holiday_flg                      int64
air_genre_name                  object
air_area_name                   object
latitude                       float64
longitude                      float64
merge_on                datetime64[ns]
visit_date_air                  object
reserve_visitors_air           float64
hpg_store_id                    object
visit_date_hpg                  object
reserve_visitors_hpg           float64
visitors                       float64
day                              int64
month                            int64
year                             int64
dayofweek                        int64
dayofyear                        int64
earlist_open_date               object
closed_flag                      int64
dtype: object

In [235]:
# clean up intermediate columns
full.drop(columns = ['merge_on','visit_date_air', 'visit_date_hpg'], inplace =True)

In [236]:
full.head()

,air_store_id,calendar_date,day_of_week,holiday_flg,air_genre_name,air_area_name,latitude,longitude,reserve_visitors_air,hpg_store_id,reserve_visitors_hpg,visitors,day,month,year,dayofweek,dayofyear,earlist_open_date,closed_flag
0,air_ba937bf13d40fb24,2016-01-01,Friday,1,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN,NaN,NaN,NaN,1,1,2016,4,1,2016-01-13,2
1,air_ba937bf13d40fb24,2016-01-02,Saturday,1,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN,NaN,NaN,NaN,2,1,2016,5,2,2016-01-13,2
2,air_ba937bf13d40fb24,2016-01-03,Sunday,1,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN,NaN,NaN,NaN,3,1,2016,6,3,2016-01-13,2
3,air_ba937bf13d40fb24,2016-01-04,Monday,0,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN,NaN,NaN,NaN,4,1,2016,0,4,2016-01-13,2
4,air_ba937bf13d40fb24,2016-01-05,Tuesday,0,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN,NaN,NaN,NaN,5,1,2016,1,5,2016-01-13,2


In [237]:
full.shape

(428593, 19)

In [238]:
full.to_csv('full_1120.csv', index=False)

In [239]:
full.columns

Index(['air_store_id', 'calendar_date', 'day_of_week', 'holiday_flg',
       'air_genre_name', 'air_area_name', 'latitude', 'longitude',
       'reserve_visitors_air', 'hpg_store_id', 'reserve_visitors_hpg',
       'visitors', 'day', 'month', 'year', 'dayofweek', 'dayofyear',
       'earlist_open_date', 'closed_flag'],
      dtype='object')

In [243]:
# sanity check
full[full.air_store_id =='air_63b13c56b7201bd9']

,air_store_id,calendar_date,day_of_week,holiday_flg,air_genre_name,air_area_name,latitude,longitude,reserve_visitors_air,hpg_store_id,reserve_visitors_hpg,visitors,day,month,year,dayofweek,dayofyear,earlist_open_date,closed_flag
262636,air_63b13c56b7201bd9,2016-01-01,Friday,1,Cafe/Sweets,Fukuoka-ken Fukuoka-shi Daimyō,33.586994,130.394894,NaN,hpg_4bc649e72e2a239a,NaN,NaN,1,1,2016,4,1,2016-01-02,1
262637,air_63b13c56b7201bd9,2016-01-02,Saturday,1,Cafe/Sweets,Fukuoka-ken Fukuoka-shi Daimyō,33.586994,130.394894,NaN,hpg_4bc649e72e2a239a,NaN,12.0,2,1,2016,5,2,2016-01-02,0
262638,air_63b13c56b7201bd9,2016-01-03,Sunday,1,Cafe/Sweets,Fukuoka-ken Fukuoka-shi Daimyō,33.586994,130.394894,NaN,hpg_4bc649e72e2a239a,NaN,35.0,3,1,2016,6,3,2016-01-02,0
262639,air_63b13c56b7201bd9,2016-01-04,Monday,0,Cafe/Sweets,Fukuoka-ken Fukuoka-shi Daimyō,33.586994,130.394894,NaN,hpg_4bc649e72e2a239a,11.0,33.0,4,1,2016,0,4,2016-01-02,0
262640,air_63b13c56b7201bd9,2016-01-05,Tuesday,0,Cafe/Sweets,Fukuoka-ken Fukuoka-shi Daimyō,33.586994,130.394894,NaN,hpg_4bc649e72e2a239a,3.0,37.0,5,1,2016,1,5,2016-01-02,0
262641,air_63b13c56b7201bd9,2016-01-06,Wednesday,0,Cafe/Sweets,Fukuoka-ken Fukuoka-shi Daimyō,33.586994,130.394894,NaN,hpg_4bc649e72e2a239a,NaN,26.0,6,1,2016,2,6,2016-01-02,0
262642,air_63b13c56b7201bd9,2016-01-07,Thursday,0,Cafe/Sweets,Fukuoka-ken Fukuoka-shi Daimyō,33.586994,130.394894,NaN,hpg_4bc649e72e2a239a,NaN,49.0,7,1,2016,3,7,2016-01-02,0
262643,air_63b13c56b7201bd9,2016-01-08,Friday,0,Cafe/Sweets,Fukuoka-ken Fukuoka-shi Daimyō,33.586994,130.394894,NaN,hpg_4bc649e72e2a239a,NaN,17.0,8,1,2016,4,8,2016-01-02,0
262644,air_63b13c56b7201bd9,2016-01-09,Saturday,0,Cafe/Sweets,Fukuoka-ken Fukuoka-shi Daimyō,33.586994,130.394894,NaN,hpg_4bc649e72e2a239a,6.0,38.0,9,1,2016,5,9,2016-01-02,0
262645,air_63b13c56b7201bd9,2016-01-10,Sunday,0,Cafe/Sweets,Fukuoka-ken Fukuoka-shi Daimyō,33.586994,130.394894,NaN,hpg_4bc649e72e2a239a,NaN,58.0,10,1,2016,6,10,2016-01-02,0


In [244]:
a_rsv[a_rsv.air_store_id =='air_63b13c56b7201bd9']

,air_store_id,visit_datetime,reserve_datetime,reserve_visitors,visit_date
60019,air_63b13c56b7201bd9,2017-02-10 13:00:00,2017-02-09 14:00:00,5,2017-02-10
60020,air_63b13c56b7201bd9,2017-02-10 13:00:00,2017-02-09 14:00:00,5,2017-02-10
60679,air_63b13c56b7201bd9,2017-02-11 12:00:00,2017-02-09 15:00:00,3,2017-02-11
60680,air_63b13c56b7201bd9,2017-02-11 12:00:00,2017-02-09 15:00:00,2,2017-02-11
60681,air_63b13c56b7201bd9,2017-02-11 12:00:00,2017-02-09 15:00:00,6,2017-02-11
60699,air_63b13c56b7201bd9,2017-02-11 13:00:00,2017-02-09 14:00:00,3,2017-02-11
65978,air_63b13c56b7201bd9,2017-02-24 13:00:00,2017-02-19 23:00:00,2,2017-02-24
66533,air_63b13c56b7201bd9,2017-02-24 21:00:00,2017-02-19 23:00:00,4,2017-02-24
66640,air_63b13c56b7201bd9,2017-02-25 12:00:00,2017-02-19 23:00:00,6,2017-02-25
67575,air_63b13c56b7201bd9,2017-02-26 18:00:00,2017-02-20 00:00:00,15,2017-02-26
